In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Vehicle_Url'] != ''].reset_index(drop=True)

dict_part = {}
for i in range(len(input_)):
    if input_.loc[i, 'Vehicle_Url'] != '':
        dict_part[input_.loc[i, 'Vehicle_Url']] = {'next': True,
                                                   'crawler': 0,
                                                   'dict_vehicle': {},
                                                   'vehicle_1': '',
                                                   'vehicle_2': ''}

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        while dict_part[input_.loc[a, 'Vehicle_Url']]['next']:
            crawler_status = 'error'
            for b in range(999):
                try:
                    resp = requests.get(input_.loc[a, 'Vehicle_Url'] + '&start=' + str(dict_part[input_.loc[a, 'Vehicle_Url']]['crawler']*50) + '&num=50',
                                        headers=UA.get_User_Agent_Requests(),
                                        proxies=Proxy.get_Proxy_Requests()).text
                    
                    # = = = = = = = = = = = = = = =
                    
                    soup = BeautifulSoup(resp, 'lxml')
                    html = etree.HTML(str(soup))
                    
                    # = = = = = = = = = = = = = = =
                    
                    list_tr = html.xpath('//tr[@class="detail-app-row"]')
                    
                    # = = = = = = = = = = = = = = =

                    for tr in list_tr:
                        make = tr.xpath('./td[2]/text()')[0].strip()
                        model = tr.xpath('./td[3]/text()')[0].strip()
                        engine = tr.xpath('./td[4]/text()')[0].strip()
                        year = int(tr.xpath('./td[1]/text()')[0].strip())

                        if make not in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle']:
                            dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make] = {model: {engine: [year]}}
                        else:
                            if model not in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make]:
                                dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model] = {engine: [year]}
                            else:
                                if engine not in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model]:
                                    dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine] = [year]
                                else:
                                    dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine].append(year)

                        dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine] = sorted(list(set(dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine])))

                    dict_part[input_.loc[a, 'Vehicle_Url']]['crawler'] += 1

                    if len(html.xpath('//a[@aria-label="Next"]')) == 0:
                        dict_part[input_.loc[a, 'Vehicle_Url']]['next'] = False
                    
                    # = = = = = = = = = = = = = = =
                    
                    if not dict_part[input_.loc[a, 'Vehicle_Url']]['next']:
                        list_vehicle = []
                        for make in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle']:
                            for model in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make]:
                                list_year = []
                                for engine in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model]:
                                    list_year += dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine]
                                list_year = sorted(list(set(list_year)))

                                year_min = min(list_year)
                                year_max = max(list_year)
                                if year_min == year_max:
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                else:
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                        dict_part[input_.loc[a, 'Vehicle_Url']]['vehicle_1'] = '\n'.join(sorted(list_vehicle))
                        
                        # = = = = = = = = = = = = = = =
                        
                        list_vehicle = []
                        for make in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle']:
                            for model in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make]:
                                for engine in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model]:
                                    year_min = min(dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine])
                                    year_max = max(dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine])
                                    if year_min == year_max:
                                        if engine == '':
                                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                        else:
                                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                                    else:
                                        if engine == '':
                                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                                        else:
                                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                        dict_part[input_.loc[a, 'Vehicle_Url']]['vehicle_2'] = '\n'.join(sorted(list_vehicle))

                        # = = = = = = = = = = = = = = =
                        
                        df_temp = pd.DataFrame([{'Vehicle_Url': input_.loc[a, 'Vehicle_Url'],
                                                 'Vehicle_1': dict_part[input_.loc[a, 'Vehicle_Url']]['vehicle_1'],
                                                 'Vehicle_2': dict_part[input_.loc[a, 'Vehicle_Url']]['vehicle_2']}])

                        output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                        
                    # = = = = = = = = = = = = = = =
                    
                    crawler_status = 'ok'
                    
                    break
                except:
                    time.sleep(0.3)
                    continue
                    
            print('[' + crawler_status + '] - ' + input_.loc[a, 'Vehicle_Url'] + ' - ' + str(dict_part[input_.loc[a, 'Vehicle_Url']]['crawler']) +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
                    
            if crawler_status == 'error':
                break

        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Vehicle_Url': input_.loc[a, 'Vehicle_Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
#         print('[' + crawler_status + '] - ' + input_.loc[a, 'Vehicle_Url'] + ' - ' + str(dict_part[input_.loc[a, 'Vehicle_Url']]['crawler']) +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Vehicle_Url'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Vehicle_Url'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：393

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=138611&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：3] - [剩余数量：381] - [当前时间：10:23:56]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133433&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：4] - [剩余数量：380] - [当前时间：10:23:57]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=141535&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：379] - [当前时间：10:23:58]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133434&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：8] - [剩余数量：378] - [当前时间：10:23:58]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=132361&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：9] - [剩余数量：377] - [当前时间：10:23:59]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=141534&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：5] - [剩余数量：376] - [当前时间：10:23:59]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=134355&PartType=Fuel%20Filler%

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=132844&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：2] - [剩余数量：327] - [当前时间：10:24:21]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=75098&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：5] - [剩余数量：326] - [当前时间：10:24:21]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=52616&PartType=Fuel%20Filler%20Neck - 2
[尝试次数：8] - [剩余数量：325] - [当前时间：10:24:22]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84114&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：325] - [当前时间：10:24:23]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=73537&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：12] - [剩余数量：324] - [当前时间：10:24:23]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=73536&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：8] - [剩余数量：324] - [当前时间：10:24:23]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84292&PartType=Fuel%20Filler%20Neck - 1
[尝试

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=131161&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：272] - [当前时间：10:24:42]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133442&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：20] - [剩余数量：271] - [当前时间：10:24:42]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84445&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：29] - [剩余数量：270] - [当前时间：10:24:42]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133441&PartType=Fuel%20Filler%20Neck - 2
[尝试次数：17] - [剩余数量：269] - [当前时间：10:24:43]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=47450&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：2] - [剩余数量：268] - [当前时间：10:24:43]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=50378&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：5] - [剩余数量：267] - [当前时间：10:24:44]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=50046&PartType=Fuel%20Filler%20Neck - 1

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=73547&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：3] - [剩余数量：217] - [当前时间：10:25:04]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=144401&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：9] - [剩余数量：216] - [当前时间：10:25:05]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=126399&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：2] - [剩余数量：215] - [当前时间：10:25:05]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=73555&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：11] - [剩余数量：214] - [当前时间：10:25:05]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=86091&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：213] - [当前时间：10:25:05]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=83308&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：5] - [剩余数量：212] - [当前时间：10:25:06]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=73522&PartType=Fuel%20Filler%20Neck - 1
[尝

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=50822&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：162] - [当前时间：10:25:32]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=24272&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：4] - [剩余数量：161] - [当前时间：10:25:33]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=32624&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：5] - [剩余数量：160] - [当前时间：10:25:33]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=28319&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：12] - [剩余数量：160] - [当前时间：10:25:33]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=10093&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：12] - [剩余数量：159] - [当前时间：10:25:34]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=22209&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：2] - [剩余数量：159] - [当前时间：10:25:34]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=18043&PartType=Fuel%20Filler%20Neck - 1
[尝试

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133435&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：4] - [剩余数量：109] - [当前时间：10:25:56]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=75229&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：7] - [剩余数量：108] - [当前时间：10:25:56]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=132263&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：2] - [剩余数量：107] - [当前时间：10:25:56]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=86555&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：36] - [剩余数量：106] - [当前时间：10:25:57]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=132367&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：2] - [剩余数量：105] - [当前时间：10:25:58]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=132256&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：10] - [剩余数量：104] - [当前时间：10:25:58]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133439&PartType=Fuel%20Filler%20Neck - 

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=75262&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：55] - [当前时间：10:26:25]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=51787&PartType=Fuel%20Filler%20Neck - 4
[尝试次数：1] - [剩余数量：54] - [当前时间：10:26:25]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=73940&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：5] - [剩余数量：54] - [当前时间：10:26:26]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=51787&PartType=Fuel%20Filler%20Neck - 5
[尝试次数：2] - [剩余数量：53] - [当前时间：10:26:27]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=90708&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：53] - [当前时间：10:26:27]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=67521&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：52] - [当前时间：10:26:28]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=90734&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：3] - 

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=97836&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：7] - [当前时间：10:26:52]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=83314&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：2] - [剩余数量：6] - [当前时间：10:26:53]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=73523&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：5] - [当前时间：10:26:53]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=75339&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：4] - [当前时间：10:26:53]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=87691&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：3] - [当前时间：10:26:54]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=86079&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：1] - [剩余数量：2] - [当前时间：10:26:54]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84180&PartType=Fuel%20Filler%20Neck - 1
[尝试次数：3] - [剩余数量：